In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from typing import Type
from torch import nn
from torch.optim import optimizer
import rasterio
import zipfile
from matplotlib import pyplot as plt
import datetime
from torchvision import transforms as transforms
import os

# --- GPU selection --- #
gpus = 2 # slot number (e.g., 3), no gpu use -> write just ' '
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpus)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
lidar_image = rasterio.open('../Data/N12/N12_lidar.tif').read()
lidar_array = np.array(lidar_image)

lidar_1n_image = rasterio.open('../Data/N12/N12_lidar_1n.tif').read()
lidar_1n_array = np.array(lidar_1n_image)

lidar_nt_image = rasterio.open('../Data/N12/N12_lidar_nt.tif').read()
lidar_nt_array = np.array(lidar_nt_image)

RGB2020_image = rasterio.open('../Data/N12/N12_RGB2020.tif').read()
RGB2020_array = np.array(RGB2020_image)

train_array = np.stack([lidar_array, lidar_1n_array, lidar_nt_array]).squeeze()
train_array = np.concatenate((train_array,RGB2020_array))
target_image = rasterio.open('../Data/N12/N12_newlc.tif').read()
target_array = np.array(target_image, dtype=int).squeeze()

target_array = np.where(target_array == 1, 0, target_array)
target_array = np.where(target_array == 2, 1, target_array)
target_array = np.where(target_array == 7, 2, target_array)
target_array = np.where(target_array == 8, 3, target_array)
target_array = np.where(target_array == 9, 4, target_array)
target_array = np.where(target_array == 10, 5, target_array)
target_array = np.where(target_array == 11, 6, target_array)

In [3]:
data_array = train_array
patch_size=40

data = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size), data_array.shape[0], patch_size, patch_size))
for i in range(0,data_array.shape[1]//patch_size):
    for j in range(0,data_array.shape[2]//patch_size):
        data[data_array.shape[1]//patch_size*i+j,:,:,:] = data_array[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size] #고치기
        
        #self.data =np.append(self.data, np.zeros(((1, data_array.shape[0], patch_size, patch_size))), axis = 0)
        #self.data = np.concatenate(self.data, np.zeros(((1, data_array.shape[0], patch_size, patch_size))))

label = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size),data_array.shape[0]+1))
print(label.shape)

for k in range(0,data_array.shape[1]//patch_size):
    for l in range(0,data_array.shape[2]//patch_size):
        label[data_array.shape[1]//patch_size*k+l,:] = np.bincount(target_array[k*patch_size:(k+1)*patch_size, l*patch_size:(l+1)*patch_size].reshape(-1), minlength=7)/(patch_size*patch_size)

(3600, 7)


In [5]:
simple_transform = transforms.Compose(
    transforms=[transforms.ToTensor(),
    transforms.Normalize(mean=1, std=0.5)]
)

In [7]:
data_transformed = simple_transform(data[0,:,:,:])

In [26]:
random_arr = np.random.random((6,40,40))
foo_transform = transforms.Compose(
    transforms=[transforms.Normalize(mean=1.0, std=0.5)]
)

In [25]:
data_tensor = torch.as_tensor(data).float()
data_tensor = foo_transform(data_tensor)

In [43]:
now = datetime.datetime.now()
path = '../Data/N12/Model/'
os.makedirs(os.path.join(path,f'{now.year}.{now.month}.{now.day}/', 'tmp/'), exist_ok=True)

In [44]:
now.hour

13

In [45]:
'RESULT_{0:0=2d}.{1:0=2d}'.format(now.hour, now.minute)

'RESULT_13.38'

In [4]:
import numpy as np
rand = np.random.permutation(100)
rand.shape
rand = np.random.randn(3,4)
rand.shape

(3, 4)

In [7]:
data = np.random.randn(6,2400,2400)
for i in range(2):
    rotated_data = np.rot90(data, k=i+1, axes=(-2, -1))
    print(rotated_data.shape)
    data = np.concatenate((data, rotated_data), axis=0)

(6, 2400, 2400)
(12, 2400, 2400)


In [2]:
train_array = np.load('../Data/N12/np/train_array.npy')

In [1]:
def calc_pad(i:int)->int:
    x = i
    for j in range(4):
        x = x*2
        x = x-4
    return x

In [7]:
def calc(i:int)->int:
    x = i
    for j in range(4):
        x = x*2
        x = x+4
    return x

In [5]:
for i in range(4000):
    x = calc_pad(i)
    if 2400%x == 0:
        print(f'Value:{i} | Output:{x}')

Value:0 | Output:-60
Value:3 | Output:-12
Value:4 | Output:4
Value:5 | Output:20
Value:10 | Output:100


In [9]:
for i in range(200):
    x = calc(i)
    print(f'Value:{i} | Output:{x}')

Value:0 | Output:60
Value:1 | Output:76
Value:2 | Output:92
Value:3 | Output:108
Value:4 | Output:124
Value:5 | Output:140
Value:6 | Output:156
Value:7 | Output:172
Value:8 | Output:188
Value:9 | Output:204
Value:10 | Output:220
Value:11 | Output:236
Value:12 | Output:252
Value:13 | Output:268
Value:14 | Output:284
Value:15 | Output:300
Value:16 | Output:316
Value:17 | Output:332
Value:18 | Output:348
Value:19 | Output:364
Value:20 | Output:380
Value:21 | Output:396
Value:22 | Output:412
Value:23 | Output:428
Value:24 | Output:444
Value:25 | Output:460
Value:26 | Output:476
Value:27 | Output:492
Value:28 | Output:508
Value:29 | Output:524
Value:30 | Output:540
Value:31 | Output:556
Value:32 | Output:572
Value:33 | Output:588
Value:34 | Output:604
Value:35 | Output:620
Value:36 | Output:636
Value:37 | Output:652
Value:38 | Output:668
Value:39 | Output:684
Value:40 | Output:700
Value:41 | Output:716
Value:42 | Output:732
Value:43 | Output:748
Value:44 | Output:764
Value:45 | Output:780
V